In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pytesseract
from PIL import Image, ImageFilter
import cv2
from collections import Counter
import os
import time
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def download_captcha(captcha_url, driver):
    # Tạo một session mới
    session = requests.Session()
    
    # Lấy cookies từ Selenium WebDriver
    selenium_cookies = driver.get_cookies()
    
    # Thêm cookies vào session
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    # Headers cho request
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }
    
    # Gửi request để tải ảnh captcha
    response = session.get(captcha_url, headers=headers)
    if response.status_code == 200:
        captcha_path = "captcha_image.png"
        with open(captcha_path, "wb") as f:
            f.write(response.content)
        print("Captcha đã được lưu vào file captcha_image.png")
        return captcha_path
    else:
        print(f"Lỗi khi tải captcha: {response.status_code}")
        return None

def preprocess_captcha(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding (binarization)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    
    # Optionally, apply noise removal (median filter)
    denoised = cv2.medianBlur(binary, 3)
    
    # Save the preprocessed image for debugging
    preprocessed_path = "preprocessed_captcha.png"
    cv2.imwrite(preprocessed_path, denoised)
    
    return preprocessed_path

def read_captcha(captcha_path):
    # Preprocess the captcha image
    preprocessed_path = preprocess_captcha(captcha_path)
    
    # Read the preprocessed image with pytesseract
    captcha_text = pytesseract.image_to_string(Image.open(preprocessed_path), config="--psm 7")
    
    return captcha_text.strip()

def run_multiple_captcha_attempts(captcha_path,attempts=10):
    results = []
    
    for i in range(attempts):
        # Read and process the captcha
        captcha_text = read_captcha(captcha_path)
        results.append(captcha_text)
        # print(f"Extracted CAPTCHA (Attempt {i + 1}): {captcha_text}")

    # Determine the most repeated result
    most_common = Counter(results).most_common(1)
    if most_common:
        print(f"\nMost repeated CAPTCHA result: {most_common[0][0].replace(" ", "")}")
    
    return most_common[0][0].replace(" ", "")
def get_cookie(user_name, password):
    while True:  # Vòng lặp để thử đăng nhập liên tục cho đến khi thành công
        # Cấu hình Selenium WebDriver với chế độ headless
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Chế độ headless
        options.add_argument("--no-sandbox")  # Khắc phục lỗi sandbox trên Linux
        options.add_argument("--disable-dev-shm-usage")  # Tăng giới hạn bộ nhớ chia sẻ
        options.add_argument("--disable-gpu")  # Tắt GPU (không cần thiết trên server không có UI)
        options.add_argument("--window-size=1920x1080")  # Đặt kích thước màn hình để tránh lỗi định vị phần tử

        # Khởi tạo driver
        driver = webdriver.Chrome(options=options)

        try:
            # Mở trang đăng nhập
            url = "https://dichvucong.moit.gov.vn/Login.aspx"
            driver.get(url)

            # Chờ các phần tử cần thiết hiển thị
            wait = WebDriverWait(driver, 10)

            # Nhập username
            username_field = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_txtLoginName"]')))
            username_field.send_keys(user_name)

            # Nhập password
            password_field = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_txtPassword"]')
            password_field.send_keys(password)

            # Lấy URL của captcha từ Selenium
            captcha_image = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_imgCaptcha"]')))
            captcha_src = captcha_image.get_attribute("src")
            print(f"Captcha URL: {captcha_src}")

            # Tải ảnh captcha bằng session từ Selenium WebDriver
            captcha_path = download_captcha(captcha_src, driver)

            # Đọc mã captcha bằng Tesseract OCR
            captcha_code = run_multiple_captcha_attempts(captcha_path,attempts=10)
            print(f"Mã captcha OCR: {captcha_code}")

            # Nhập mã captcha
            captcha_field = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_txtCaptcha"]')
            captcha_field.send_keys(captcha_code)

            # Nhấn nút đăng nhập
            login_button = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_btnLogin"]')
            login_button.click()

            # Chờ tải xong sau khi đăng nhập
            time.sleep(5)

            # Kiểm tra thông báo lỗi captcha (nếu có)
            try:
                captcha_notice = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_lblMsg"]').text
                if captcha_notice:
                    print(f"Thông báo: {captcha_notice}")
                    continue  # Quay lại đầu vòng lặp nếu đăng nhập thất bại
            except:
                pass

            # Kiểm tra đăng nhập thành công hay không
            if "Dashboard" in driver.page_source:
                print("Đăng nhập thành công!")

                # Đăng nhập sang ecosys
                next_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_grdViewDefault"]/tbody/tr[2]/td[4]/a')))
                next_button.click()

                # Lấy cookie
                cookie = f'{driver.get_cookies()[3]["name"]}={driver.get_cookies()[3]["value"]}'

                # Đóng driver và trả về cookie
                driver.quit()
                if os.path.exists("captcha_image.png"):
                    os.remove("captcha_image.png")
                return cookie
            else:
                print("Đăng nhập thất bại. Thử lại...")

        except Exception as e:
            print(f"Lỗi xảy ra: {e}")

        finally:
            # Đóng trình duyệt nếu không thành công
            driver.quit()
            if os.path.exists("captcha_image.png"):
                os.remove("captcha_image.png")


In [2]:
cookie = get_cookie("sedovina","sedovina")


Captcha URL: https://dichvucong.moit.gov.vn/GeneralCaptchaHandler.ashx?iwidth=120&iheight=50&t=638707349874899491
Captcha đã được lưu vào file captcha_image.png

Most repeated CAPTCHA result: frifh
Mã captcha OCR: frifh
Thông báo: Mã xác thực không đúng
Captcha URL: https://dichvucong.moit.gov.vn/GeneralCaptchaHandler.ashx?iwidth=120&iheight=50&t=638707349989880307
Captcha đã được lưu vào file captcha_image.png

Most repeated CAPTCHA result: np8uzft
Mã captcha OCR: np8uzft
Thông báo: Mã xác thực không đúng
Captcha URL: https://dichvucong.moit.gov.vn/GeneralCaptchaHandler.ashx?iwidth=120&iheight=50&t=638707350137996382
Captcha đã được lưu vào file captcha_image.png

Most repeated CAPTCHA result: ysz7nn
Mã captcha OCR: ysz7nn
Đăng nhập thành công!


'Bifrost_SessionID=twuawcul4vh4opkhzqkdqj00'